In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11fad83d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11faa5930>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

human_message = HumanMessage(content="Hi, I am Ram, I am an AI Engineer.")
model.invoke([human_message])

AIMessage(content="Hello Ram, it's nice to meet you! \n\nI'm Gemma, an AI assistant. What can I do for you today? Are you working on any interesting projects?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 21, 'total_tokens': 62, 'completion_time': 0.074545455, 'prompt_time': 0.001338962, 'queue_time': 0.246685895, 'total_time': 0.075884417}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--23f2f5ed-d990-4bbc-8d0d-2cc6ef289304-0', usage_metadata={'input_tokens': 21, 'output_tokens': 41, 'total_tokens': 62})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, I am Ram, I am an AI Engineer."), 
        AIMessage(content="Hello Ram, how can I assist you today?"),
        HumanMessage(content="I need help with my AI project.")
    ]
)

AIMessage(content="I'd love to help! To give you the best assistance, could you tell me more about your project? For example:\n\n* **What type of AI project is it?** (e.g., natural language processing, computer vision, machine learning, robotics)\n* **What are you trying to achieve with your project?**\n* **What specific challenges are you facing?**\n* **What tools and technologies are you using?**\n* **Do you have any code snippets you'd like me to review?**\n\n\nThe more information you provide, the better I can understand your needs and offer relevant help.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 49, 'total_tokens': 181, 'completion_time': 0.24, 'prompt_time': 0.001713663, 'queue_time': 0.247372798, 'total_time': 0.241713663}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1eea6ebd-d434-4838-bc68-c0b28a13d1a6-0', usag

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [
         HumanMessage(content="Hi, I am Ram, I am an AI Engineer."), 
    ],
    config=config
)

In [8]:
response.content

"Hello Ram! It's nice to meet you.\n\nI'm Gemma, an AI assistant. What can I do for you today? Are you working on any interesting projects?\n"

In [9]:
with_message_history.invoke(
    [
         HumanMessage(content="what's my name"), 
    ],
    config=config
)

AIMessage(content='Your name is Ram. You told me at the beginning of our conversation! 😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 74, 'total_tokens': 105, 'completion_time': 0.056363636, 'prompt_time': 0.002368604, 'queue_time': 0.248031773, 'total_time': 0.05873224}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a0d3681c-bf3b-4a83-a859-fb96791d1add-0', usage_metadata={'input_tokens': 74, 'output_tokens': 31, 'total_tokens': 105})

In [10]:
## change the session id

config2={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [
         HumanMessage(content="My name is john?"), 
    ],
    config=config2
)
response.content

"That's a nice name! \n\nIs there anything I can help you with, John?\n"

In [11]:

response=with_message_history.invoke(
    [
         HumanMessage(content="What's my name?"), 
    ],
    config=config2
)
response.content

"You told me your name is John!  😊 \n\nIs there something else you'd like to know or talk about?\n"

### Prompt Templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated.
First, let's add in a system message with some custom instructions (but still taking messages as input). Next,
we'll add in more input besides just the messages.

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt|model

In [13]:
chain.invoke({
    "messages": [HumanMessage(
        content="Hello, My name is Ram"
    )]
})

AIMessage(content="Hello Ram! It's nice to meet you.\n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 32, 'total_tokens': 58, 'completion_time': 0.047272727, 'prompt_time': 0.001799362, 'queue_time': 0.257969598, 'total_time': 0.049072089}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2d8aa1df-21d9-436a-9a5c-b4758041fd80-0', usage_metadata={'input_tokens': 32, 'output_tokens': 26, 'total_tokens': 58})

In [14]:
with_message_history = RunnableWithMessageHistory(
    chain, get_session_history
)

In [15]:
config={"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke({"messages": [HumanMessage(content="Hello, My name is Ram")]}, config=config)
response.content

"Hello Ram, it's nice to meet you!\n\nWhat can I do for you today? 😊  \n\n"

In [16]:
### Add more complexity
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt|model

response=chain.invoke({"messages": [HumanMessage(content="Hello, My name is Ram")], "language":"Tamil"}, )
response.content

'வணக்கம், ராம்! \n\nநான் உங்கள் கேள்விகளுக்கு உதவும் ஒரு வணக்கம். என்ன தகவல் தேடுகிறீர்கள்? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [17]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [18]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages": [HumanMessage(content="Hello, My name is Ram")], "language":"Tamil"}, 
    config=config
    )
response.content

'வணக்கம், ரமேஷ்!  நான் உதவக்கூடிய உதவிக்குழு. என்ன கேட்க விரும்புகிறீர்கள்? \n\n'

In [19]:
response=with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name")], "language":"Tamil"}, 
    config=config
    )
response.content

'உங்கள் பெயர் ரமேஷ். \n\n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [25]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
    )
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [26]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What math problem i have asked?")],
        "language":"English" 
    }
)
response.content

'You asked "what is 2 + 2".  \n\n\n\nIs there another problem you\'d like to try? 😊\n'

In [27]:
## Lets wrap this with Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [29]:
response=with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name")], "language":"English"}, 
    config=config
    )
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it! 😊  \n\n"

In [30]:
response=with_message_history.invoke(
    {"messages": [HumanMessage(content="What math problem i have asked?")], "language":"English"}, 
    config=config
    )
response.content

"Please provide me with the math problem! I'm ready to help. 😊  \n"